In [1]:
### Set app credentials
import dotenv
dotenv.load_dotenv()

True

In [2]:
### Initialize the chatbot model
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [3]:
### Prive a template
from langchain import hub

prompt = hub.pull('hwchase17/openai-functions-agent')

In [5]:
### Create Document Vector store
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("attention.pdf")
data = loader.load()

### Create Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
docs = text_splitter.split_documents(data)

### Embed and Store chunks
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

In [6]:
### Create a retriever tool
from langchain.tools.retriever import create_retriever_tool

retriever = vectorstore.as_retriever()
retriever_tool = create_retriever_tool(
    retriever=retriever, 
    name="document_search",
    description="Search for information in the document. If you dont find any awnsers dont give one."
)

In [7]:
### Create a wiki tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wiki_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)

In [8]:
tools = [retriever_tool, wiki_tool]

In [9]:
### Create an agent
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

# Create the agent excecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [10]:
### Examples

In [11]:
agent_executor.invoke({"input":"What's the point of this document?"})



> Entering new AgentExecutor chain...

Invoking: `document_search` with `{'query': 'point of this document'}`


References

described in section 3.2.

best models from the literature.
1 Introduction

the approach we take in our model.The document mentions references, the best models from the literature, and the approach taken in the model. If you have a specific question about the document, feel free to ask!

> Finished chain.


{'input': "What's the point of this document?",
 'output': 'The document mentions references, the best models from the literature, and the approach taken in the model. If you have a specific question about the document, feel free to ask!'}

In [12]:
agent_executor.invoke({"input":"What approach was taken in the model?"})



> Entering new AgentExecutor chain...

Invoking: `document_search` with `{'query': 'approach in the model'}`


the approach we take in our model.

in order for the model to make use of the

3.2.3 Applications of Attention in our Model

the competitive models.The search did not return relevant information about the approach taken in the model. I can attempt to gather information from other sources.

> Finished chain.


{'input': 'What approach was taken in the model?',
 'output': 'The search did not return relevant information about the approach taken in the model. I can attempt to gather information from other sources.'}

In [13]:
agent_executor.invoke({"input":"What is the Model Architecture of this?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Model Architecture`


Page: Architectural model
Summary: An architectural model is a type of scale model made to study aspects of an architectural design or to communicate design intent. They are made using a variety of maThe term "Model Architecture" can refer to different types of models, including architectural models used to study aspects of architectural design. However, without more specific context, it's unclear which model architecture you are referring to. Could you please provide more details or specify the type of model architecture you are asking about?

> Finished chain.


{'input': 'What is the Model Architecture of this?',
 'output': 'The term "Model Architecture" can refer to different types of models, including architectural models used to study aspects of architectural design. However, without more specific context, it\'s unclear which model architecture you are referring to. Could you please provide more details or specify the type of model architecture you are asking about?'}

In [14]:
agent_executor.invoke({"input":"Can you tell me more about The Transformer model architecture?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `Transformer (machine learning model)`


Page: Transformer (deep learning architecture)
Summary: A transformer is a deep learning architecture developed by Google and based on the multi-head attention mechanism, proposed in a 2017 paper "AttThe Transformer model architecture is a deep learning architecture developed by Google. It is based on the multi-head attention mechanism, which was proposed in a 2017 paper. The Transformer model has been widely used in natural language processing tasks and has demonstrated state-of-the-art performance in various language-related applications. If you would like to know more details, feel free to ask!

> Finished chain.


{'input': 'Can you tell me more about The Transformer model architecture?',
 'output': 'The Transformer model architecture is a deep learning architecture developed by Google. It is based on the multi-head attention mechanism, which was proposed in a 2017 paper. The Transformer model has been widely used in natural language processing tasks and has demonstrated state-of-the-art performance in various language-related applications. If you would like to know more details, feel free to ask!'}